# Graph Mode Computations
In previous section of the course we investigate the properties and characteristics of **Graph Mode** and **Eager Mode** in TensorFlow. We saw that although most of times they behave same, there are cases where they behave differently like writing `print` statements or debugging. \
In this assignment we will investigate other cases:

In [1]:
import tensorflow as tf
import numpy as np

## 1. Lists
Python list is poorly supported in graph mode. In particular, when the list is modified inside or outside the `@tf.function` method.\
**Run the next two cells to see the unexpected behaviour.**


In [2]:
l = []

@tf.function
def f(x):
    for i in x:
        l.append(i + 1)

In [3]:
f(tf.constant([1, 2, 3]))
print(l)

[<tf.Tensor 'while/add:0' shape=() dtype=int32>]


Let's rewrite this function with the use of `TensorArray` which is a data structure for dynamic arrays.
use its `write` and `stack` method to rewrite the above function.\
https://www.tensorflow.org/api_docs/python/tf/TensorArray#write

https://www.tensorflow.org/api_docs/python/tf/TensorArray#stack

In [18]:
@tf.function
def f(x):
    ta = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    for i in range(len(x)):
        # Write your code here
        ta = ta.write(i, x[i]+1)
    return ta.stack()

In [19]:
f(tf.constant([1, 2, 3]))

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([2, 3, 4], dtype=int32)>

In [20]:
print(f(tf.constant([1, 2, 3])))

tf.Tensor([2 3 4], shape=(3,), dtype=int32)


## 2. Tensor Objects and Numpy Arrays
With eager execution, Numpy operations can take tf.Tensor as parameters.\
run the following cells

In [21]:
def np_multiply(a, b):
    return np.multiply(a, b)

In [22]:
a_tensor = tf.constant([5])
b_tensor = tf.constant([10])
np_multiply(a_tensor, b_tensor)

array([50], dtype=int32)

What about vice versa?\
Write an eager function that gets two numpy array and by using `tf.math.multiply` multiply these arrays.

In [23]:
@tf.function
def tf_multiply(a, b):
    #write your code here
    return tf.math.multiply(a, b)

In [24]:
a_array = np.array([5])
b_array = np.array([10])
tf_multiply(a_array, b_array)

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([50])>

now convert there two functions into Graph functions and run again:

In [25]:
np_multiply_tf_func = tf.function(np_multiply) #Write your code here
tf_multiply_tf_func = tf.function(tf_multiply) #Write your code here

In [27]:
np_multiply_tf_func(a_tensor, b_tensor)

NotImplementedError: in user code:

    File "/tmp/ipython-input-284224191.py", line 2, in np_multiply  *
        return np.multiply(a, b)

    NotImplementedError: Cannot convert a symbolic tf.Tensor (a:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported.


As you noticed we can't use `numpy` operations inside graph functions and we must use their `TensorFlow` equivalents.\
Also TensorFlow has a new API which implemented the numpy functions:
https://www.tensorflow.org/guide/tf_numpy

In [28]:
tf_multiply_tf_func(a_array, b_array)

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([50])>

## 3. Tensor Assignment
Let's write an eager function that gets a one dimensional Tensor and assing `1` to its first element

In [29]:
def assign_one(x):
    x[1] = 1
    return x

In [30]:
x = tf.constant([5, 6, 7])
assign_one(x)

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

You can see that TensorFlow `Tensor` object does not support item assignment in contrast with numpy array or python lists

But there is a way to do this in eager mode:
1. create a numpy array from the tensor by calling `tensor.numpy()
2. assign the value to numpy array
3. create a constant tensor from the numpy array

let's write it

In [37]:
def assign_one_new(x):
    x_numpy = x.numpy()
    #Write your code here
    x_numpy[1] = 1
    return tf.constant(x_numpy)

In [38]:
x = tf.constant([5, 6, 7])
assign_one_new(x)

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([5, 1, 7], dtype=int32)>

Now convert this function to a Graph function

In [39]:
assign_one_graph = tf.function(assign_one_new) #Write your code here

In [40]:
x = tf.constant([5, 6, 7])
assign_one_graph(x)

AttributeError: in user code:

    File "/tmp/ipython-input-693387072.py", line 2, in assign_one_new  *
        x_numpy = x.numpy()

    AttributeError: 'SymbolicTensor' object has no attribute 'numpy'


You can see that there is no way to assign a value to a `Tensor` in graph mode, you should consider it in your implementations.
You can use `tf.Variable` instead of Tensors for assignments:
https://www.tensorflow.org/api_docs/python/tf/Variable#assign

In [58]:
def assign_one_new2(x):
    x[1].assign(1)
    return x

In [59]:
assign_one_graph2 = tf.function(assign_one_new2)

y = tf.Variable([5, 6, 7])
assign_one_graph2(y)

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([5, 1, 7], dtype=int32)>

## Conclusion
In this assignment we have investigated some differences of Graph mode and Eager mode in TensorFlow, specially we saw that working with Python Lists and Numpy Arrays have some difficulties with Graph Mode